In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths to your dataset (update this)
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# Set image size and batch size
pic_size = 48  # FER-2013 images are 48x48 pixels
batch_size = 64

## Image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)


# Ensure the generators repeat the data
train_generator = train_datagen.flow_from_directory(
    base_path + "/train",           
    target_size=(pic_size, pic_size), 
    color_mode="rgb",  
    batch_size=batch_size,            
    class_mode='categorical',         
    shuffle=Trueee).repeat()

validation_generator = validation_datagen.flow_from_directory(
    base_path + "/validation",
    target_size=(pic_size, pic_size),
    color_mode="rgb",  
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
).repeat()



Found 28821 images belonging to 7 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'repeat'

In [7]:
# VGG16 Model - Use pre-trained weights from ImageNet
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(pic_size, pic_size, 3))

# Freeze the VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

# Create the model
model = models.Sequential([
    vgg_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # 7 classes in FER-2013
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Class weights (to handle imbalance in classes)
class_weight_dict = {
    0: 1.0,  # Angry
    1: 9.0,  # Disgust
    2: 1.0,  # Fear
    3: 0.5,  # Happy
    4: 0.8,  # Neutral
    5: 0.8,  # Sad
    6: 1.0   # Surprise
}

# Callbacks
checkpoint = ModelCheckpoint(
    'model_weights_vgg16.keras',  
    monitor='val_accuracy',  
    verbose=1,  
    save_best_only=True,  
    mode='max'   
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [9]:
# Train the model with the correct steps_per_epoch and validation_steps
history = model.fit(
    x=train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator) // batch_size,  # Adjusted
    validation_steps=len(validation_generator) // batch_size,  # Adjusted
    callbacks=[checkpoint, early_stop]
)

Epoch 1/20


/Users/pratiksha/.pyenv/versions/3.9.19/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 817ms/step - accuracy: 0.1958 - loss: 1.8264
Epoch 1: val_accuracy improved from -inf to 0.31772, saving model to model_weights_vgg16.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 457s 1s/step - accuracy: 0.1958 - loss: 1.8262 - val_accuracy: 0.3177 - val_loss: 1.7451
Epoch 2/20


2024-11-26 13:27:23.576332: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/pratiksha/.pyenv/versions/3.9.19/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-26 13:27:23.589133: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


AttributeError: 'NoneType' object has no attribute 'items'